In [1]:
from geocube.api.core import make_geocube
from pathlib import Path
import geopandas as gpd
import dask_geopandas

In [2]:
ddf = dask_geopandas.read_file(Path('/home/geofelpave/Documents/6_Test_Area/data/glymphs/GLHYMPS/GLHYMPS.shp'), npartitions=4)
# ddf.to_crs(4326)
ddf.columns

Index(['OBJECTID_1', 'IDENTITY_', 'logK_Ice_x', 'logK_Ferr_', 'Porosity_x',
       'K_stdev_x1', 'OBJECTID', 'Descriptio', 'XX', 'YY', 'ZZ', 'AA', 'DD',
       'Shape_Leng', 'GUM_K', 'Prmfrst', 'Shape_Le_1', 'Shape_Area',
       'Transmissi', 'COUNT', 'AREA_1', 'MEAN', 'STD', 'geometry'],
      dtype='object')

In [3]:
test = ddf[['logK_Ferr_', 'geometry']].copy()
test['k'] = 10**(test['logK_Ferr_'] / 100)
test['Ksat'] = test['k'] * 100000000000
test

,logK_Ferr_,geometry,k,Ksat
npartitions=4,,,,
0,int32,geometry,float64,float64
450811,...,...,...,...
901622,...,...,...,...
1352433,...,...,...,...
1803241,...,...,...,...


In [4]:
df_glhymps = test.compute()

In [5]:
df_glhymps_new = df_glhymps[['Ksat', 'geometry']].copy()
# save to shapefile
df_glhymps_new.to_file(Path('/home/geofelpave/Documents/6_Test_Area/data/glymphs/GLHYMPS/GLHYMPS_KSat.shp')) 

: 

: 

In [11]:
df_grid = gpd.read_file(Path('/home/geofelpave/Documents/1_PhD_SharePoint/LFPV - PhD - Documents/00_PhD_main/006_GitHub/0065_odc_hydro/shetran_data/active_data/final_mask.shp'))
df_grid_new = df_grid.to_crs(df_glhymps.crs)

In [16]:
df_grid_new

,intersect,SHETRAN_ID,geometry
0,0,-9999,"POLYGON ((-300472.404 5228354.090, -291692.954..."
1,1,0,"POLYGON ((-300384.383 5225503.256, -291614.841..."
2,1,0,"POLYGON ((-300296.590 5222649.187, -291536.953..."
3,1,0,"POLYGON ((-300209.039 5219791.894, -291459.288..."
4,1,0,"POLYGON ((-300121.723 5216931.372, -291381.836..."
...,...,...,...
145,1,0,"POLYGON ((-221464.449 5199827.042, -212782.076..."
146,0,-9999,"POLYGON ((-221465.687 5196943.951, -212792.988..."
147,0,-9999,"POLYGON ((-221466.931 5194057.647, -212803.864..."
148,0,-9999,"POLYGON ((-221468.265 5191168.115, -212814.785..."


: 

In [5]:
df_glhymps.crs

<Derived Projected CRS: PROJCS["Cylindrical_Equal_Area",GEOGCS["WGS 84",DA ...>
Name: Cylindrical_Equal_Area
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Cylindrical Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
df_grid_new.crs

<Derived Projected CRS: PROJCS["Cylindrical_Equal_Area",GEOGCS["WGS 84",DA ...>
Name: Cylindrical_Equal_Area
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Cylindrical Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
intersect = df_glhymps.overlay(df_grid_new, how='intersection')

In [15]:
intersect

,logK_Ferr_,k,Ksat,intersect,SHETRAN_ID,geometry
0,-1650,3.162278e-17,0.000003,1,0,"POLYGON ((-268119.818 5227203.114, -268549.444..."
1,-1250,3.162278e-13,0.031623,1,0,"POLYGON ((-271214.004 5227020.114, -270706.505..."
2,-1300,1.000000e-13,0.010000,1,0,"POLYGON ((-270173.401 5225586.905, -270285.780..."
3,-1250,3.162278e-13,0.031623,1,0,"POLYGON ((-271585.747 5227598.327, -271504.331..."
4,-1650,3.162278e-17,0.000003,1,0,"MULTIPOLYGON (((-273426.364 5228156.302, -2731..."
...,...,...,...,...,...,...
949,-1250,3.162278e-13,0.031623,0,-9999,"MULTIPOLYGON (((-290756.891 5185549.009, -2906..."
950,-1052,3.019952e-11,3.019952,0,-9999,"MULTIPOLYGON (((-299190.052 5186378.922, -2989..."
951,-1135,4.466836e-12,0.446684,0,-9999,"MULTIPOLYGON (((-290616.791 5185427.666, -2907..."
952,-1250,3.162278e-13,0.031623,1,0,"MULTIPOLYGON (((-241604.291 5191173.662, -2420..."
